In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, UpSampling2D

In [ ]:
input_shape = (256, 256, 3)
kerSize = 3

In [ ]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    "/mnt/s/Proyects/dataset/train/outdoor",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="training"
)

In [ ]:
val_data = tf.keras.preprocessing.image_dataset_from_directory(
    "/mnt/s/Proyects/dataset/val",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="validation"
)

In [ ]:
model = tf.keras.Sequential()

# Encoder
model.add(Conv2D(32, kernel_size=3, strides=2, padding='same',input_shape=input_shape))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Conv2D(64, kernel_size=3, strides=2, padding='same'))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Conv2D(128, kernel_size=3, strides=2, padding='same'))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Conv2D(256, kernel_size=3, strides=2, padding='same'))
model.add(BatchNormalization())
model.add(ReLU())

# Decoder
model.add(Conv2D(256, kernel_size=3, strides=1, padding='same'))
model.add(BatchNormalization())
model.add(ReLU())
model.add(UpSampling2D(size=(2, 2)))
model.add(Conv2D(128, kernel_size=3, strides=1, padding='same'))
model.add(BatchNormalization())
model.add(ReLU())
model.add(UpSampling2D(size=(2, 2)))
model.add(Conv2D(64, kernel_size=3, strides=1, padding='same'))
model.add(BatchNormalization())
model.add(ReLU())
model.add(UpSampling2D(size=(2, 2)))
model.add(Conv2D(32, kernel_size=3, strides=1, padding='same'))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Conv2D(1, kernel_size=3, strides=1, padding='same', activation='sigmoid'))

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mean_squared_error')

In [ ]:
model.fit(train_data, epochs=100, validation_data=val_data)

In [ ]:
import tensorflow as tf
import cv2
import numpy as np

# Cargar la imagen
img = cv2.imread("/mnt/s/Proyects/dataset/val/outdoor/scene_00022/scan_00193/00022_00193_outdoor_000_020.png")

# Escalar la imagen
img = cv2.resize(img, (256, 256))

# Normalizar la imagen
img = np.float32(img) / 255.0

# Agregar una dimensión adicional para representar el lote (batch)
img = np.expand_dims(img, axis=0)
img = np.expand_dims(img, axis=-1)

# Cargar el modelo entrenado
# model = tf.keras.models.load_model('path/to/model.h5')

# Realizar la predicción de profundidad
depth = model.predict(img)

# Imprimir la profundidad predicha
print(depth)


In [ ]:
print(len(depth))
print(len(depth[0]))
print(len(depth[0][0]))
print(len(depth[0][0][0]))
print(depth[0][0][0][0])

for i in range(128):
    for j in range(128):
        depth[0][i][j][0] = depth[0][i][j][0]*255.0

In [ ]:
from PIL import Image
import numpy as np

w, h = 512, 512
data = np.zeros((h, w, 3), dtype=np.uint8)
data[0:256, 0:256] = [255, 0, 0] # red patch in upper left
img = Image.fromarray(data, 'RGB')
img.save('my.png')
img.show()